## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-15-16-48-23 +0000,nyt,"Angered by Israel’s Attack in Qatar, Arab Lead...",https://www.nytimes.com/2025/09/15/world/middl...
1,2025-09-15-16-44-00 +0000,bbc,Starmer aide resigns after explicit texts abou...,https://www.bbc.com/news/articles/ckgy79yr74do...
2,2025-09-15-16-42-44 +0000,bbc,Malawi set to vote for next president as cost-...,https://www.bbc.com/news/articles/ckgy3g7zpl3o...
3,2025-09-15-16-39-57 +0000,nyt,"In Giant Deals, U.A.E. Got Chips, and Trump Te...",https://www.nytimes.com/2025/09/15/us/politics...
4,2025-09-15-16-37-42 +0000,nyt,"Turning Against Vaccines, America Is a Global ...",https://www.nytimes.com/2025/09/15/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2307/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,35
148,kirk,22
147,charlie,18
99,china,13
337,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
205,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...,92
238,2025-09-14-19-31-04 +0000,nypost,Eric Trump: Assassination of Charlie Kirk has ...,https://nypost.com/2025/09/14/us-news/eric-tru...,90
83,2025-09-15-13-44-49 +0000,nypost,Chilling new video shows Charlie Kirk suspect ...,https://nypost.com/2025/09/15/us-news/video-sh...,88
69,2025-09-15-14-16-39 +0000,nyt,U.S.-China Deal to Avert TikTok Ban May be Clo...,https://www.nytimes.com/2025/09/15/technology/...,74
189,2025-09-15-02-00-00 +0000,wsj,A bullet casing found after the killing of Cha...,https://www.wsj.com/politics/policy/fascist-en...,71


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
205,92,2025-09-14-23-51-33 +0000,nypost,Trump blasts ‘the left’ and announces ‘major i...,https://nypost.com/2025/09/14/us-news/trump-bl...
40,47,2025-09-15-15-15-00 +0000,wsj,Canadian factories notched the biggest rise in...,https://www.wsj.com/articles/canada-factory-an...
83,42,2025-09-15-13-44-49 +0000,nypost,Chilling new video shows Charlie Kirk suspect ...,https://nypost.com/2025/09/15/us-news/video-sh...
75,40,2025-09-15-14-03-11 +0000,nypost,"Bessent says US, China have reached ‘framework...",https://nypost.com/2025/09/15/business/us-chin...
44,38,2025-09-15-15-04-49 +0000,nypost,Top GOP foreign-policy Rep. Michael McCaul won...,https://nypost.com/2025/09/15/us-news/texas-re...
23,32,2025-09-15-15-48-34 +0000,nyt,Elon Musk Buys $1 Billion in Tesla Stock as Bo...,https://www.nytimes.com/2025/09/15/business/el...
127,32,2025-09-15-10-26-35 +0000,nypost,Zohran Mamdani shares eerie similarities to fo...,https://nypost.com/2025/09/15/us-news/zohran-m...
12,30,2025-09-15-16-05-52 +0000,nypost,Trump renews controversial idea for axing quar...,https://nypost.com/2025/09/15/business/trump-r...
22,30,2025-09-15-15-53-08 +0000,nyt,2 Men Arrested After Incendiary Device Is Foun...,https://www.nytimes.com/2025/09/15/us/utah-exp...
71,28,2025-09-15-14-14-04 +0000,nypost,Charlie Kirk shooting is ‘the death of free sp...,https://nypost.com/2025/09/15/media/charlie-ki...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
